# [모듈 1.1] CodeCommit 리파지토리 생성

이 노트북은 CodeCommit 리파지토리를 생성하고, codecommit 폴더를 리파지토리에 Push 하는 노트북 입니다.

## 참고 자료
- [Quick guide to start using AWS CodeCommit via AWS CLI](https://dev.to/tiamatt/quick-guide-to-start-using-aws-codecommit-via-aws-cli-1jg5)
- 개발자 가이드
    - [Setup for HTTPS users using Git credentials](https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-gc.html)
    - [Create an AWS CodeCommit repository](https://docs.aws.amazon.com/codecommit/latest/userguide/how-to-create-repository.html#how-to-create-repository-cli)
    



# 1. 환경 확인

In [61]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
%%sh 
git --version

git version 2.37.1


In [63]:
%%sh
aws --version

aws-cli/1.24.10 Python/3.8.12 Linux/5.10.102-99.473.amzn2.x86_64 botocore/1.26.10


# 2. 파라미터 로딩

In [64]:
%store -r code_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)

import json
print (json.dumps(code_pipeline_serving_dict, indent=2))

repository_name = code_pipeline_serving_dict["serving_code_repo_name"]

{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/codebuild-gsmoon",
  "project_prefix": "CodePipeline-Serving-NCF",
  "region": "us-east-1",
  "account_id": "057716757052",
  "serving_code_repo_name": "ncf-serving",
  "code_build_project_name": "ncf-serving-sm-pipeline",
  "bucket": "sagemaker-us-east-1-057716757052",
  "code_pipeline_name": "ncf-serving-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "endpoint_name": "ncf-codepipeline-endpoint",
  "branch_name": "master"
}


In [65]:
source_code = 'codecommit'


# 3.리파지토리 생성

In [66]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit create-repository --repository-name $repository_name \
--repository-description "My inference repository" 

{
    "repositoryMetadata": {
        "accountId": "057716757052",
        "repositoryId": "0d2debd8-014e-4f0d-9151-a05bee294dff",
        "repositoryName": "ncf-serving",
        "repositoryDescription": "My inference repository",
        "lastModifiedDate": 1668946057.016,
        "creationDate": 1668946057.016,
        "cloneUrlHttp": "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-serving",
        "cloneUrlSsh": "ssh://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-serving",
        "Arn": "arn:aws:codecommit:us-east-1:057716757052:ncf-serving"
    }
}


# 4. 리파지토리 로컬에 클로닝
### 아래를 위의 결과에서 카피해서 대체 해주세요.
#### clone_url =   "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/mlops-ncf-serving-pipeline"


In [67]:
clone_url = "https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-serving"
# clone_url = "<Clone URL>"

In [68]:
%%sh -s {clone_url} {repository_name}

clone_url=$1
repository_name=$2

rm -rf $repository_name

git clone $clone_url 

Cloning into 'ncf-serving'...


# 5. 소스 코드 복사

In [69]:
%%sh -s {repository_name} {source_code}
repository_name=$1
source_code=$2

cp -r $source_code/* $repository_name
ls $repository_name

codebuild-buildspec-221120.yml
codebuild-buildspec.yml
CONTRIBUTING.md
img
LICENSE
pipelines
README.md
sagemaker-pipelines-project.ipynb
setup.cfg
setup.py
tests
tox.ini


# 6. 코드 파일을 리파지토리에 푸시

In [70]:
%%sh -s {repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m"Add new files"
git push origin master


/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/5_sm-serving-codepipeline/ncf-serving
[master (root-commit) f2c15db] Add new files
 Committer: EC2 Default User <ec2-user@ip-172-16-47-148.ec2.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 32 files changed, 2084 insertions(+)
 create mode 100644 CONTRIBUTING.md
 create mode 100644 LICENSE
 create mode 100644 README.md
 create mode 100644 codebuild-buildspec-221120.yml
 create mode 100644 codebuild-buildspec.yml
 create mode 100644 img/pipeline-full.png
 create mode 100644 pipelines/.ipynb_checkpoints/run_pipeline-checkpoint.py
 create mode 100644 pipeli

To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/ncf-serving
 * [new branch]      master -> master
